In [8]:
import json
import os
from types import SimpleNamespace

import aiohttp
import cohere
import yaml

co = cohere.Client("KVfAFBdgNHFDMjksbmkogAuO2eYiCmEEkTppKjzj")

async def async_summarize(**options):
    headers = {
        "Authorization": "BEARER " + co.api_key,
        "Content-Type": "application/json",
    }
    async with aiohttp.ClientSession() as session:
        async with session.post(
            co.api_url + "/summarize", headers=headers, json=options
        ) as resp:
            resp = await resp.json()
            x = json.loads(json.dumps(resp), object_hook=lambda d: SimpleNamespace(**d))
            return x


co.async_summarize = async_summarize  # add async summarize call to cohere client


async def make_history(self, message, num_historic_msgs=20):
    history = []
    messages = [
        message
        async for message in message.channel.history(
            limit=num_historic_msgs, before=message
        )
    ]
    for historic_msg in messages:
        name = historic_msg.author.name
        line = ""

        if bool(historic_msg.content):
            if (
                historic_msg.author.name == self.user.name
            ):  # disregard previous chatbot summaries in the new summary
                continue
            line += historic_msg.clean_content
        if line:
            history.insert(0, {name: line})
    return "\n".join(
        [
            f"{list(chat_turn.keys())[0]}: {list(chat_turn.values())[0]}"
            for chat_turn in history
        ]
    )

def prepare_prompt(self, history, message):
    return (
        open("prompts/" + CONFIGS["PROMPT_FILE"])
        .read()
        .format(
            history=history.strip(),
            message=message,
        )
    )

async def summarize(self, prompt):
    response = await co.async_summarize(
        text=prompt,
        model="summarize-xlarge",
        length="medium",
        extractiveness="medium",
    )
    return response.summary

async def on_message(self, message):
    if (self.user.mentioned_in(message) and message.author != self.user):
        async with message.channel.typing():
            history = await self.make_history(message, num_historic_msgs=20)
            prompt = self.prepare_prompt(
                history, f"{message.author.name}: {message.clean_content}"
            )
            summary = await self.summarize(prompt)
            await message.channel.send(summary)
            return
